# test Klein-Gordon Model


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torch
import numpy as np
import gc

Import local packages


In [ ]:


from src.utils.logger import Logging
from src.nn.pde import klein_gordon_operator
from src.data.klein_gordon_dataset import generate_training_dataset, f, u

Set various constant variables: model path, name, etc. We choose the best performing models from each architecture.


In [ ]:
TANH_NORM_DIST = "./model/a1/klein_tanh_normal_a1.pth"
BSPLINE_DIST = "./model/a1/klein_bspline_a1.pth"
CHEBTSHEV_DIST = "./model/a1/klein_chebyshev_a1.pth"
GRBF_DIST = "./model/a1/klein_grbf_a1.pth"
JACOBI_DIST = "./model/a1/klein_jacobi_a1.pth"
TANH_PARAM_DIST = "./model/a2/klein_tanh_trainable_a2.pth"
FOURIER_DIST = "./model/a1/klein_fourier_a1.pth"


model_path_list = {
    "tanh": TANH_NORM_DIST,
    "param_tanh": TANH_PARAM_DIST,
    "grbf": GRBF_DIST,
    "bspline": BSPLINE_DIST,
    "chebyshev": CHEBTSHEV_DIST,
    "jacobi": JACOBI_DIST,
    "fourier": FOURIER_DIST,
}

In [7]:
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/klein_gordon")
logger = Logging(TEST_CHECKPOINT_PATH)
result_dir = logger.get_output_dir()

In [ ]:
number_of_points = 100

alpha = -1.0
beta = 0.0
gamma = 1.0
k = 3


[_, _, res_sampler] = generate_training_dataset("cpu")


dom_coords = np.array([[0.0, 0.0], [1.0, 1.0]])

number_of_points = 50
t = np.linspace(dom_coords[0, 0], dom_coords[1, 0], number_of_points)[:, None]
x = np.linspace(dom_coords[0, 1], dom_coords[1, 1], number_of_points)[:, None]


t, x = np.meshgrid(t, x)
X_star = torch.tensor(
    np.hstack((t.flatten()[:, None], x.flatten()[:, None])), dtype=torch.float32
).to("cpu")

u_star = u(X_star)
f_star = f(X_star, alpha, beta, gamma, k)

t, x = X_star[:, 0:1], X_star[:, 1:2]


logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

logger.print(f"Result directory {model_dirname}")

INFO:src.utils.logger:Result directory /home/ubuntu/afrah/code/pinn_learnable_activation/testing_checkpoints/klein_gordon/2024-10-05_17-04-47-060064


In [ ]:
SOLVER_TO_MODULE = {
    "grbf": "src.nn.grbf",
    "jacobi": "src.nn.jacobi",
    "bspline": "src.nn.bspline",
    "chebyshev": "src.nn.chebyshev",
    "MLP": "src.nn.tanh_parameterized",
    "tanh": "src.nn.tanh",
    "fourier": "src.nn.fourier",
}

for activation, model_path in model_path_list.items():
    logger.print(f"MODEL_PATH {model_path}")
    state = torch.load(model_path)
    config = state.get("config", {})
    solver = config.get("solver")

    model_activation_name = config.get("activation")
    model_architecture = config.get("network")
    loss_history = state.get("loss_history")
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")
        model = PINNKAN(model_architecture, model_activation_name).to("cpu")

    model.load_state_dict(state["model_state_dict"])
    logger.print(f"Model {activation} loaded from {model_path}")
    logger.print(f"activation:  , {config.get('activation')}")
    logger.print(f"solver:  , {config.get('solver')}")
    logger.print(f"problem:  , {config.get('problem')}")
    logger.print(f"number of iterations:  , {len(next(iter(loss_history.values())))}")
    logger.print(f"batch size:  , {config.get('batch_size')}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")

    [u_pred, f_pred] = klein_gordon_operator(model, t, x, 0, 1)
    if u_pred.is_cuda:
        u_pred = u_pred.cpu()
        f_pred = f_pred.cpu()

    logger.print(
        f"Model {activation} with iterations: {len(loss_history[next(iter(loss_history))])}"
    )

    X_analytic = X_star.detach().numpy()
    u_analytic = u_star.detach().numpy()
    f_analytic = f_star.detach().numpy()
    u_pred = u_pred.detach().numpy()
    f_pred = f_pred.detach().numpy()

    error_u = (
        np.linalg.norm(u_analytic - u_pred, 2) / np.linalg.norm(u_analytic, 2)
    ) * 100.0
    error_f = (
        np.linalg.norm(f_analytic - f_pred, 2) / np.linalg.norm(f_analytic + 1e-9, 2)
    ) * 100.0

    logger.print("Relative L2 error_u: {:.2e}".format(error_u))
    logger.print("Relative L2 error_f: {:.2e}".format(error_f))

    for key in loss_history:
        logger.print("Final loss %s: %e" % (key, loss_history[key][-1]))

    logger.print("******************************\n")

    del model
    torch.cuda.empty_cache()
    gc.collect()

INFO:src.utils.logger:MODEL_PATH ../../model/a1/klein_tanh_normal_a1.pth
INFO:src.utils.logger:Model tanh loaded from ../../model/a1/klein_tanh_normal_a1.pth
INFO:src.utils.logger:activation:  , None
INFO:src.utils.logger:solver:  , tanh
INFO:src.utils.logger:problem:  , klein_gordon
INFO:src.utils.logger:number of iterations:  , 60001
INFO:src.utils.logger:batch size:  , 128
INFO:src.utils.logger:network:  , [2, 30, 30, 30, 1]
INFO:src.utils.logger:term loss weights :  , [50.0, 50.0, 1.0]


INFO:src.utils.logger:Model tanh with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 1.58e+00
INFO:src.utils.logger:Relative L2 error_f: 5.08e-01
INFO:src.utils.logger:Final loss lbcs: 5.982671e-05
INFO:src.utils.logger:Final loss linitial: 4.500597e-06
INFO:src.utils.logger:Final loss lphy: 1.867867e-01
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH ../../model/a2/klein_tanh_trainable_a2.pth
INFO:src.utils.logger:Model param_tanh loaded from ../../model/a2/klein_tanh_trainable_a2.pth
INFO:src.utils.logger:activation:  , tanh
INFO:src.utils.logger:solver:  , MLP
INFO:src.utils.logger:problem:  , None
INFO:src.utils.logger:number of iterations:  , 60001
INFO:src.utils.logger:batch size:  , 128
INFO:src.utils.logger:network:  , [2, 400, 400, 400, 1]
INFO:src.utils.logger:term loss weights :  , [50.0, 50.0, 1.0]
INFO:src.utils.logger:Model param_tanh with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 1.85e+01
INFO:src.ut